In [1]:
## Install Selenium library and setting

!apt-get update &> /dev/null
!apt install chromium-chromedriver &> /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin &> /dev/null
!pip install selenium &> /dev/null

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless') # hide chrome browser
options.add_argument('--no-sandbox') # bypass OS security model
options.add_argument('--disable-dev-shm-usage') # overcome limited resource problems
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")

In [2]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import HTTPError
import pandas as pd
import re
import time

import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# **Collect article using Selenium**

In [3]:
## 크롤링을 하기 위한 홈페이지 페이지 구조 확인(링크 내에서 키워드, 반복 페이지 구조 및 위치)
# https://www.washingtonpost.com/newssearch/?query=wolfensohn%20world%20bank&btn-search=&sort=Relevance&datefilter=All%20Since%202005&contenttype=Article&startat=0&spellcheck#top # startat=숫자 이후의 &spellcheck#top 는 홈페이지 자체 옵션과 같은 항목이라 크롤링에서는 무시해도 무관함
# https://www.washingtonpost.com/newssearch/?query=wolfensohn%20world%20bank&btn-search=&sort=Relevance&datefilter=All%20Since%202005&contenttype=Article&startat=20&spellcheck#top

KEYWORD = 'wolfensohn world bank'

LINK = f'https://www.washingtonpost.com/newssearch/?query={KEYWORD}&btn-search=&sort=Relevance&datefilter=All%20Since%202005&contenttype=Article&spellcheck&startat='
LINK


'https://www.washingtonpost.com/newssearch/?query=wolfensohn world bank&btn-search=&sort=Relevance&datefilter=All%20Since%202005&contenttype=Article&spellcheck&startat='

In [4]:
## Make def to collect WP articles

TITLES = []
TITLES_LINKS = []
DATES = []

def get_link_from_news_title(url, article_num):
    NUM = int(article_num/20 + 1) 
    for i in range(0, NUM):
        n = 20*i
        URL = LINK + str(n)
        DRIVER = webdriver.Chrome('chromedriver', options=options)
        DRIVER.implicitly_wait(3) #페이지가 뜨는데 걸리는 시간 3초
        DRIVER.get(URL)

        SEARCH_RESULTS = DRIVER.find_elements_by_css_selector('div.pb-feed-headline.ng-scope')
        for RESULT in SEARCH_RESULTS:
            TITLES.append(RESULT.find_element_by_tag_name('a').text) # title
            TITLES_LINKS.append(RESULT.find_element_by_tag_name('a').get_attribute('href')) # title link
                
        ARTICLE_DATE = DRIVER.find_elements_by_css_selector('div.pb-feed-byline.ng-binding')
        for DATE in ARTICLE_DATE:
            DATES.append(DATE.find_element_by_tag_name('span').text) # date

    if n % 40 == 0:
        time.sleep(2)
        print(f'{n}nd webpage completed')


In [5]:
## Collect articles (구글 코랩의 경우 일정 시간이 지나면 연결이 풀려버리기 때문에 너무 많은 기사 개수를 가져올 경우 중간에 작업이 풀려버릴 수 있음)
get_link_from_news_title(LINK, 84) # 워싱턴 포스트에서 검색 키워드로 전체 기사가 몇 개가 나오는지를 확인한 뒤(위에서 LINK 변수) 숫자 입력

print(len(TITLES))
print(len(TITLES_LINKS))
print(len(DATES))

80nd webpage completed
83
83
83


In [6]:
## Make dataframe and look at results

WP = pd.DataFrame(list(zip(TITLES, TITLES_LINKS, DATES)), columns =['Title', 'Url', 'Date'])
WP.head()

,Title,Url,Date
0,"James D. Wolfensohn, who led World Bank through tumultuous decade, dies at 86",https://www.washingtonpost.com/local/obituaries/james-wolfensohn-dead/2020/11/25/be7c0100-2f48-11eb-bae0-50bb17126614_story.html,"Nov 25, 2020"
1,"Vernon E. Jordan Jr., lawyer and D.C. political power broker, dies at 85",https://www.washingtonpost.com/local/obituaries/vernon-jordan-dead/2021/03/02/b633fb82-4b79-11ea-9b5c-eac5b16dafaa_story.html,"Mar 2, 2021"
2,The Finance 202: Fed's Powell warns Washington will remain on economic clean-up duty for foreseeable future,https://www.washingtonpost.com/politics/2020/12/17/finance-202-fed-powell-warns-washington-will-remain-economic-clean-up-duty-foreseeable-future/,"Dec 17, 2020"
3,Community deaths,https://www.washingtonpost.com/local/obituaries/community-deaths/2020/08/24/32329e56-e642-11ea-bc79-834454439a44_story.html,"Aug 24, 2020"
4,Trump pick for World Bank chief spent years criticizing its mission,https://www.washingtonpost.com/business/economy/trump-pick-for-world-bank-chief-spent-years-criticizing-its-mission/2019/02/05/fe291824-2969-11e9-b2fc-721718903bfc_story.html,"Feb 5, 2019"


In [7]:
from google.colab import drive # mount Google Drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [8]:
WP.to_csv('/content/drive/MyDrive/소셜빅데이터분석_2021/WP_links.csv', encoding='utf-8-sig', index_label = False, index = False)

# **Gathering article text from pre-collected data**

In [9]:
## Read saved data file
# WP = pd.read_csv("/content/drive/MyDrive/소셜빅데이터분석_2021/WP_links.csv", encoding='utf-8', index_col=[0])

In [10]:
## Make variables from collected data

TITLES = WP['Title']
TITLES_LINKS = WP['Url']
DATES = WP['Date']

Washinton Post의 기사 본문 수집의 경우, 기사 본문의 div class 구조가 teaser와 reminder로 분리되어 있어 기존의 방법이 적용되지 않아 상위 tag인 article-body 단위에서 수집 후 태그를 제거하는 방법을 거침

* 현재 워싱턴 포스트는 대략 2009~2010년 이전 기사들이 현재와 다른 홈페이지 구조로 만들어져 여러 가지 경우의 수를 고려해야 함

In [11]:
CONTENTS = []

def get_text(url):
    for i in range(0, len(url)):
        try:
            REQ = urllib.request.Request(url[i],
                                        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})
            SOURCE = urllib.request.urlopen(REQ)
            SOUP = BeautifulSoup(SOURCE, features='lxml', from_encoding='utf-8')
            ITEM = str(SOUP.select('div.article-body')) # contents
            URL_CONTENT = re.sub('<.+?>', '', ITEM)
            URL_CONTENT = re.sub('\n', '', URL_CONTENT)
            CONTENTS.append(URL_CONTENT)

            # 2005~2010년 기사들의 경우 html 구조가 최신 기사들과 달라 별도의 방식으로 처리함
            # 해당 기사들은 일종의 header와 body 형태로 구분되어 있어 두 개를 따로 크롤링한 뒤 합치는 과정을 거친다

            if URL_CONTENT == '[]': #아무 것도 없으면 문자열을 만들어 무언가 넣을 준비를 해라.....
                ITEM = str(SOUP.select('div#article_body > p')) # 헤더 부분 id는 #으로 표시...
                URL_CONTENT = re.sub('<.+?>', '', ITEM) # +는 조합을 의미한다....p, div 상관없이 소스코드 모두 없애주는...
                URL_CONTENT = re.sub('\n', '', URL_CONTENT) # 문단은 엔터로 구별되어 있으니, 없애라....

                ITEM1 = str(SOUP.select('div#body_after_content_column > p')) # 바디 부분 id는 #으로 표시...
                URL_CONTENT1 = re.sub('<.+?>', '', ITEM1) # +는 조합을 의미한다....p, div 상관없이 소스코드 모두 없애주는...
                URL_CONTENT1 = re.sub('\n', '', URL_CONTENT1) # 문단은 엔터로 구별되어 있으니, 없애라....

                CONTENTS[i] = URL_CONTENT + " " + URL_CONTENT1
        
        except URLError: # 일부 기사들의 경우 지나치게 자주 조회가 되면 서버에서 조회를 막아버리기 때문에 이를 패싱하는 코드를 입력한다
            CONTENTS.append('110 Server not connected')

        if i % 40 == 0:
            time.sleep(2) # 기사 40개당 2초 쉬기 / 기사 수집 중에 ip가 차단될 경우 횟수를 줄여서 sleep timing의 간격을 짧게 할 것
            print(f'{i}nd item is collected')
    
    print(len(CONTENTS))

In [12]:
get_text(TITLES_LINKS)

0nd item is collected
40nd item is collected
80nd item is collected
83


In [ ]:
print(len(CONTENTS))

WP_DATA = pd.DataFrame(list(zip(TITLES, TITLES_LINKS, DATES, CONTENTS)), columns =['Title', 'Url', 'Date', 'Text'])
WP_DATA.head()

83


Title  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
WP_DATA.to_csv('/content/drive/MyDrive/소셜빅데이터분석_2021/WP_DB.csv', encoding='utf-8-sig', index_label = False, index = False)

# **Preprocessing**

* converting all letters to lower or upper case
* converting numbers into words or removing numbers
* removing punctuations, accent marks and other diacritics
* removing white spaces
* expanding abbreviations
* removing stop words, sparse terms, and particular words
* etc...

In [ ]:
## world bank 크롤링 과정에서만 추가될 수 있는 두 가지 요소 제거

#ARTICLE_PRE = [re.sub('Support our journalism. Subscribe today.arrow-right', '', x) for x in WP_DATA['Text']] # Support our journalism. Subscribe today.arrow-right 라는 광고가 html 태그 내 숨어있음
#ARTICLE_PRE = [re.sub('AD+', ' ', x) for x in ARTICLE_PRE] # AD(AD) 라는 무의미한 태그가 숨어있음 

In [ ]:
#stop_words = stopwords.words('english')

In [ ]:
## lowercases, tokenizes, de-accents

"""
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc), min_len=3) if word not in stop_words] for doc in texts]
"""
#text_no_stops = remove_stopwords(ARTICLE_PRE)

In [ ]:
## lemmatization

"""
def lemmatization(sentences, allowed_postags=['NOUN', 'ADJ', 'VERB']): 
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in sentences:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
"""
#text_lemmatized = lemmatization(text_no_stops, allowed_postags=['NOUN'])

In [ ]:
#text_lemmatized[0:4]